In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12501,2024-11-21,Eua Nba,00:00,Golden State Warriors,Atlanta Hawks,1.30,3.72,238.5,1.84,2.00,-8.5,2.02,1.83,IDvyyHMU,0.769231,0.268817,0.543478,0.500000,0.038048,0.4,0.6,NaN,NaN,294.338,121.257975,0.411968,426.318,354.848695,0.832357,186.470,415.668,201.60,270.32,0.0,0.0,0.0,0.0,0.681752,0.058926,0.069792,1.97,0.394,0.761421,0.647530,0.7,0.052470,9.12,1.824,1.491228,0.427096,0.3,-0.127096
12502,2024-11-21,Eua Nba,00:00,Phoenix Suns,New York Knicks,2.67,1.51,221.5,1.80,2.03,4.5,1.97,1.89,p0tTxwhI,0.374532,0.662252,0.555556,0.492611,0.036783,0.6,0.6,NaN,NaN,234.594,83.680876,0.356705,178.928,26.275504,0.146850,161.190,177.822,174.64,219.78,0.0,0.0,0.0,0.0,0.392461,0.084927,0.029310,0.66,0.132,12.651515,0.644532,0.6,-0.044532,-1.74,-0.348,-1.465517,0.600149,0.4,-0.200149
12503,2024-11-21,Eua Nba,00:30,Los Angeles Clippers,Orlando Magic,2.46,1.59,204.5,1.93,1.93,3.5,1.97,1.89,8hYzuTYo,0.406504,0.628931,0.518135,0.518135,0.035435,0.2,0.4,NaN,NaN,289.254,97.738365,0.337898,209.386,78.636872,0.375559,206.544,317.992,277.44,346.32,0.0,0.0,0.0,0.0,0.303794,0.000000,0.029310,2.55,0.510,2.862745,0.547184,0.6,0.052816,-5.00,-1.000,-0.590000,0.423618,0.1,-0.323618
12504,2024-11-21,Eua Nba,21:00,Charlotte Hornets,Detroit Pistons,1.88,1.78,219.5,1.81,1.85,-1.5,2.01,1.68,GIyqw73b,0.531915,0.561798,0.552486,0.540541,0.093713,0.4,0.2,NaN,NaN,249.688,42.569825,0.170492,266.328,48.750316,0.183046,387.318,326.492,303.85,344.40,0.0,0.0,0.0,0.0,0.038640,0.015456,0.126474,1.25,0.250,3.520000,0.444136,0.5,0.055864,1.26,0.252,3.095238,0.383686,0.4,0.016314
12505,2024-11-21,Eua Nba,21:30,Toronto Raptors,Minnesota Timberwolves,3.52,1.33,224.5,1.80,1.86,5.5,2.01,1.68,OnZVXpvm,0.284091,0.751880,0.555556,0.537634,0.035971,1.0,0.8,NaN,NaN,477.026,126.566308,0.265324,139.578,22.487893,0.161113,400.678,162.956,480.77,118.58,1.0,0.0,0.0,0.0,0.638583,0.023184,0.126474,1.20,0.240,10.500000,0.398583,0.5,0.101417,-1.76,-0.352,-0.937500,0.613059,0.3,-0.313059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12594,2024-11-21,Eua Ncaa,23:30,Washington State,East. Washington,1.16,4.31,159.5,1.80,1.86,-12.5,2.01,1.68,xdjXrDJE,0.862069,0.232019,0.555556,0.537634,0.094088,0.8,0.6,NaN,NaN,138.736,36.730580,0.264752,528.730,563.474744,1.065714,121.090,500.320,98.10,1463.00,0.0,1.0,0.0,0.0,0.814401,0.023184,0.126474,2.35,0.470,0.340426,0.663750,1.0,0.336250,-1.53,-0.306,-10.816993,0.539920,0.6,0.060080
12595,2024-11-21,Europa Eurocopa Feminina,16:30,Proteas Voulas F,Crvena zvezda F,14.70,1.01,142.5,1.83,1.83,20.5,2.01,1.71,Ei0htzBN,0.068027,0.990099,0.546448,0.546448,0.058126,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,560.56,188.80,1.0,0.0,0.0,0.0,1.232373,0.000000,0.114049,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12596,2024-11-21,Lituânia Nkl,14:00,BC Olimpas Palanga,Telsiai,1.74,1.93,153.5,1.83,1.83,-2.5,2.02,1.67,jaMwixUQ,0.574713,0.518135,0.546448,0.546448,0.092847,0.6,0.8,NaN,NaN,120.280,15.277173,0.127013,113.992,20.069821,0.176063,126.436,132.884,131.04,113.46,0.0,0.0,0.0,0.0,0.073215,0.000000,0.134139,-0.32,-0.064,-11.562500,0.709805,0.8,0.090195,-3.78,-0.756,-1.230159,0.694246,0.5,-0.194246
12597,2024-11-21,Rússia Copa Da Rússia,12:00,Samara F,Novosibirsk F,1.11,18.50,139.5,1.83,1.83,-14.5,1.9

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Atlanta Hawks,238.5,1.84,0.8787,Over
1,00:00,Eua Nba,Phoenix Suns,New York Knicks,221.5,1.80,0.7885,Over
2,00:30,Eua Nba,Los Angeles Clippers,Orlando Magic,204.5,1.93,0.9634,Over
3,22:00,Eua Nba,San Antonio Spurs,Utah Jazz,221.5,1.83,0.8210,Over
